In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from datetime import datetime


In [2]:
driver = webdriver.Chrome()
driver.get("https://www.worldsbk.com/en/calendar")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('li', {'class': 'col-xs-12 col-sm-6 col-md-3 col-lg-3'})

driver.quit()

# <li class="col-xs-12 col-sm-6 col-md-3 col-lg-3" style="opacity: 1;">


In [ ]:
schedule_details

In [10]:
days = [] 
end_days = []
months = []
end_months = []
titles = [] 
descs = []
circuits = []
time = []
start_hours = []
start_minutes = []
end_hours = []
end_minutes = []

year = datetime.now().year

round_counter = 1


month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'February': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'April': 4,
        'May': 5,
        'Jun': 6,
        'June': 6,
        'Jul': 7,
        'July': 7,
        'Aug': 8,
        'August': 8,
        'Sep': 9,
        'September': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'November': 11,
        'Dec': 12,
        'DEC': 12,
        'December': 12
    }




In [11]:
address = 'https://www.worldsbk.com'
driver = webdriver.Chrome()
for i in schedule_details:
    if i.find(class_='calendar-round-item banner-calendar'):
        continue
    else:
        link = i.find('a', {'class': 'track-link'})['href']        

        driver.get(address + link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Set the 'active' class to the 'My time' tab. This allows the times scraped to be in my local time, 
        #     otherwise would need to extract event time zones for the schedule later
        my_time_tab = soup.find('span', class_='tab', attrs={'data-tab-id': 'my_time'})
        my_time_tab['class'] = ['tab', 'active']

           # Find all divs with class 'timeIso'
        time_iso_divs = soup.find_all('div', class_='timeIso')

        for div in time_iso_divs:
            # Extract the start time from the div
            start_time = div.find('div', attrs={'data_ini': True})['data_ini'].strip()
            # Extract the session name from the div
            session_name = div.find('div', class_='cat-session').text.strip()

             # Check if the session name contains 'WorldSBK Race 1' or 'WorldSBK Race 2'
            if session_name == 'WorldSBK - Race 1' or session_name == 'WorldSBK - Race 2':
                time, date = start_time.split('T')[1].rstrip('+'), start_time.split('T')[0]

                month, day = int(date.split('-')[1]), int(date.split('-')[2])
                months.append(month)
                days.append(day)

                start_h, start_m = int(time.split(':')[0]), int(time.split(':')[1])
                start_hours.append(start_h)
                start_minutes.append(start_m)

                end_h, end_m = start_h + 1, start_m
                if end_h >= 24:
                    end_h -= 24
                    end_day = day + 1
                else:
                    end_day = day
                end_days.append(end_day)
                end_months.append(month)
                end_hours.append(end_h)
                end_minutes.append(end_m)

                circuit_info = soup.find('div', class_='info-circuit')
                circuit_name = circuit_info.contents[0].strip()
                circuits.append(circuit_name)

                temp_title = 'WSBK Round' + ' ' + str(round_counter)
                round_counter += 1
                titles.append(temp_title)

                desc_p1 = soup.find('p').text
                full_desc = desc_p1 + ' ' + session_name
                descs.append(full_desc)



driver.quit()        

     
                
                
        



    



        
       
#     for i in descs:
#         if '6h' in i:
#             end_hour = start_hour + 6
#             end_hours.append(end_hour)
#         else:
#             end_hour = start_hour + 4
#             end_hours.append(end_hour)

#     else:
#         pass

In [ ]:
br

In [12]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [13]:
WSBK = Calendar()
WSBK.add('prodid', '-//My calendar product//example.com//')
WSBK.add('version', '2.0')
uid = 110

In [14]:
 
for title, desc, circuit, day, month, s_h, s_m, end_day, end_month, e_h, e_m in zip(titles, descs, circuits, days, months, start_hours, start_minutes, end_days, end_months, end_hours, end_minutes):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, month, day, s_h, s_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(year, end_month, end_day, e_h, e_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(year, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    WSBK.add_component(ievent)

In [15]:
WSBK

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'WSBK Round 1''), 'DESCRIPTION': vText('b'Grand Ridge Brewery Australian Round WorldSBK - Race 1''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000002178CF86B50>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000002178CF86880>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000002178CF86910>, 'LOCATION': vText('b'''), 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'WSBK Round 2''), 'DESCRIPTION': vText('b'Grand Ridge Brewery Australian Round WorldSBK - Race 2''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000002178CF86580>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000002178CF86FD0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000002178CF86970>, 'LOCATION': vText('b'''), 'UID': vText('b'111'')}), VEVENT({'SUMMARY': vText('b'WSBK Round 3''), 'DESCRIPTION': vText('b'Pirelli Catalunya Round WorldSBK - Race 1''), 'DTSTART': <icalenda

In [ ]:
br

In [16]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'WSBK.ics'), 'wb')
f.write(WSBK.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
